In [1]:
import random
import pandas as pd
from datetime import datetime

# Temporary workflow
# (1) Download csv with `heroku run make exportdb && make importdb`, load it here
# (2) save links to ~/Downloads/links.txt in onetab format
# (3) run this script (or otherwise modify the CSV as desired)
# (4) Upload back up with `make csv_to_s3`
# (5) Run `heroku run make importdb` to get it to production
links = pd.read_csv('data/export.csv')
links

,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet
0,0,http://moralitylab.bc.edu/wp-content/uploads/2...,Moral Realism As Moral Motivation: The Impact ...,"""People disagree about whether 'moral facts' a...",moralitylab.bc.edu,2016-12-06 05:00:00+00,2016-12-06 05:00:00+00,1.0,ethics,custom,17.0,0.0
1,1,http://www.jefftk.com/news/2013-04-01,The Unintuitive Power Laws of Giving,"""Why give globally? Why give money? Why health...",www.jefftk.com,2016-12-07 05:00:00+00,2016-12-07 05:00:00+00,1.0,giving,custom,59.0,0.0
2,2,http://calnewport.com/blog/2013/03/24/how-to-w...,How to Write Six Important Papers a Year witho...,"""I'm fascinated by people who produce a large ...",calnewport.com,2016-12-08 05:00:00+00,2016-12-08 05:00:00+00,1.0,productivity,custom,56.0,0.0
3,3,http://www.freakonomics.com/2012/10/31/an-alte...,An Alternative to Democracy,"""In Glen's voting mechanism\, every voter can ...",www.freakonomics.com,2016-12-09 05:00:00+00,2016-12-09 05:00:00+00,1.0,politicalscience,custom,29.0,0.0
4,4,http://www.wired.com/vanish/2009/11/ff_vanish2...,Writer Evan Ratliff Tried to Vanish: Here's Wh...,"""August 13\, 6:40 PM: I'm driving East out of ...",www.wired.com,2016-12-10 05:00:00+00,2016-12-10 05:00:00+00,1.0,technology,custom,92.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
77406,133941,https://statmodeling.stat.columbia.edu/2021/10...,Pundits continue to push the white-men-are-dyi...,\N,\N,2021-10-05 13:10:57.786523+00,2021-10-05 13:10:57.786523+00,NaN,\N,Gelman,81.0,0.0
77407,133942,https://www.slowboring.com/p/is-quiet-climate-...,Is quiet climate policy enough?,\N,\N,2021-10-05 13:10:57.791267+00,2021-10-05 13:10:57.791267+00,NaN,\N,Yglesias,62.0,0.0
77408,133943,https://alifeofproductivity.com/remember-burno...,Remember: Burnout and cynicism go hand-in-hand,\N,\N,2021-10-05 13:10:57.796335+00,2021-10-05 13:10:57.796335+00,NaN,\N,ALOP,47.0,0.0
77409,133945,https://twitter.com/peterwildeford/status/1445...,Apparently the most prescribed medicine right ...,\N,twitter.com,2021-10-06 01:47:36.907991+00,2021-10-06 01:47:36.907982+00,1.0,science,Custom,1.0,1.0


In [2]:
links['added'].max()

'2021-10-06 02:33:56.273501+00'

In [3]:
links[(links['aggregator'] == 'Custom') & (links['liked'] != '1') & (links['liked'] != '0') & (links['liked'] != '-1')]['id']

/Users/peterhurford/.virtualenvs/dev/lib/python3.8/site-packages/pandas/core/computation/expressions.py:68: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return op(a, b)
/Users/peterhurford/.virtualenvs/dev/lib/python3.8/site-packages/pandas/core/computation/expressions.py:68: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return op(a, b)
/Users/peterhurford/.virtualenvs/dev/lib/python3.8/site-packages/pandas/core/computation/expressions.py:68: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return op(a, b)


1299       1306
1300       1307
1301       1308
1302       1309
1303       1310
          ...  
77356    133891
77357    133892
77358    133893
77409    133945
77410    133946
Name: id, Length: 3625, dtype: int64

In [4]:
link_file = open('/Users/peterhurford/Downloads/links.txt', 'r')
new_links = link_file.readlines()

In [5]:
def new_entry(links, url, title):
    return {'id': links['id'].max() + 1,
            'url': str(url).strip().replace(',', ''),
            'title': str(title).strip().replace(',', ''),
            'summary': '\\N',
            'domain': '\\N',
            'added': str(datetime.now()),
            'modified': str(datetime.now()),
            'liked': '\\N',
            'category': '\\N',
            'aggregator': 'Custom',
            'seed': random.randint(1, 100),
            'tweet': 0}

for i, new_link in enumerate(new_links):
    if i % 100 == 0:
        print('...{}/{}'.format(i + 1, len(new_links)))
    if ' | ' in new_link:
        new_link = new_link.replace('    - ', '').replace('\n', '')
        entry = new_entry(links, new_link.split(' | ')[0], new_link.split(' | ')[1])
        links = links.append(entry, ignore_index=True)

links

...1/64


,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet
0,0,http://moralitylab.bc.edu/wp-content/uploads/2...,Moral Realism As Moral Motivation: The Impact ...,"""People disagree about whether 'moral facts' a...",moralitylab.bc.edu,2016-12-06 05:00:00+00,2016-12-06 05:00:00+00,1,ethics,custom,17.0,0.0
1,1,http://www.jefftk.com/news/2013-04-01,The Unintuitive Power Laws of Giving,"""Why give globally? Why give money? Why health...",www.jefftk.com,2016-12-07 05:00:00+00,2016-12-07 05:00:00+00,1,giving,custom,59.0,0.0
2,2,http://calnewport.com/blog/2013/03/24/how-to-w...,How to Write Six Important Papers a Year witho...,"""I'm fascinated by people who produce a large ...",calnewport.com,2016-12-08 05:00:00+00,2016-12-08 05:00:00+00,1,productivity,custom,56.0,0.0
3,3,http://www.freakonomics.com/2012/10/31/an-alte...,An Alternative to Democracy,"""In Glen's voting mechanism\, every voter can ...",www.freakonomics.com,2016-12-09 05:00:00+00,2016-12-09 05:00:00+00,1,politicalscience,custom,29.0,0.0
4,4,http://www.wired.com/vanish/2009/11/ff_vanish2...,Writer Evan Ratliff Tried to Vanish: Here's Wh...,"""August 13\, 6:40 PM: I'm driving East out of ...",www.wired.com,2016-12-10 05:00:00+00,2016-12-10 05:00:00+00,1,technology,custom,92.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
77470,134006,http://www.cleverhans.io/security/privacy/ml/2...,The challenge of verification and testing of m...,\N,\N,2021-10-05 23:00:44.806635,2021-10-05 23:00:44.806647,\N,\N,Custom,51.0,0.0
77471,134007,http://www.cleverhans.io/security/privacy/ml/2...,Is attacking machine learning easier than defe...,\N,\N,2021-10-05 23:00:44.815512,2021-10-05 23:00:44.815523,\N,\N,Custom,78.0,0.0
77472,134008,http://www.cleverhans.io/security/privacy/ml/2...,Breaking things is easy,\N,\N,2021-10-05 23:00:44.824349,2021-10-05 23:00:44.824360,\N,\N,Custom,86.0,0.0
77473,134009,https://docs.google.com/document/d/1kaHdXIqQ_7...,AID research proposals (4/2021) - Google Docs,\N,\N,2021-10-05 23:00:44.832961,2021-10-05 23:00:44.832973,\N,\N,Custom,33.0,0.0


In [6]:
links[(links['aggregator'] == 'Custom') & (links['liked'] != '1') & (links['liked'] != '0') & (links['liked'] != '-1')]

,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet
1299,1306,http://blog.beeminder.com/trust/,Trusting Your Divided Self | Beeminder Blog,\N,blog.beeminder.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,0,productivity,Custom,10.0,0.0
1300,1307,http://powlyglot.com/fi3m-review-part-1-focus/,"""What I Learned from Fluent in 3 Months\, Part...",\N,powlyglot.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,0,productivity,Custom,9.0,0.0
1301,1308,http://effective-altruism.com/ea/u8/ea_intervi...,"""EA Interview Series\, February 2016: John Sal...",\N,effective-altruism.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,0,effectivealtruism,Custom,85.0,0.0
1302,1309,https://fivethirtyeight.com/features/most-2020...,Most 2020 Candidates Have Something In Common:...,\N,fivethirtyeight.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,1,politicalscience,Custom,82.0,0.0
1303,1310,http://www.smartpassiveincome.com/backlinking-...,THE Backlinking Strategy That Works – 2014 and...,\N,www.smartpassiveincome.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,0,entrepreneurship,Custom,59.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
77470,134006,http://www.cleverhans.io/security/privacy/ml/2...,The challenge of verification and testing of m...,\N,\N,2021-10-05 23:00:44.806635,2021-10-05 23:00:44.806647,\N,\N,Custom,51.0,0.0
77471,134007,http://www.cleverhans.io/security/privacy/ml/2...,Is attacking machine learning easier than defe...,\N,\N,2021-10-05 23:00:44.815512,2021-10-05 23:00:44.815523,\N,\N,Custom,78.0,0.0
77472,134008,http://www.cleverhans.io/security/privacy/ml/2...,Breaking things is easy,\N,\N,2021-10-05 23:00:44.824349,2021-10-05 23:00:44.824360,\N,\N,Custom,86.0,0.0
77473,134009,https://docs.google.com/document/d/1kaHdXIqQ_7...,AID research proposals (4/2021) - Google Docs,\N,\N,2021-10-05 23:00:44.832961,2021-10-05 23:00:44.832973,\N,\N,Custom,33.0,0.0


In [7]:
def clean(txt):
    if not isinstance(txt, str):
        return txt
    txt = txt.replace('&nbsp;', '')
    txt = txt.replace('&ldquo;', '"')
    txt = txt.replace('&rdquo;', '"')
    txt = txt.replace('&lsquo;', '\'')
    txt = txt.replace('&rsquo;', '\'')
    txt = txt.replace('&mdash;', '-')
    txt = txt.replace('&ndash;', '-')
    for i in range(10):
        txt = txt.replace('"""', '"')
        txt = txt.replace('\'\'\'', '\'')
    txt = txt.replace('n"t', 'n\'t')
    txt = txt.replace('n""t', 'n\'t')
    return txt

links = links[links['id'].notnull()]   # Drop empty column
links['id'] = links['id'].astype(int)  # Fix float ID issue
links['seed'] = links['seed'].astype(int)
links['tweet'] = links['tweet'].apply(lambda x: 0 if str(x) == '\\N' else str(x).split('.')[0]).astype(int)
links['summary'] = links['summary'].apply(clean)
links['title'] = links['title'].apply(clean)
links = links.sort_values('id')
links

,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet
0,0,http://moralitylab.bc.edu/wp-content/uploads/2...,Moral Realism As Moral Motivation: The Impact ...,"""People disagree about whether 'moral facts' a...",moralitylab.bc.edu,2016-12-06 05:00:00+00,2016-12-06 05:00:00+00,1,ethics,custom,17,0
1,1,http://www.jefftk.com/news/2013-04-01,The Unintuitive Power Laws of Giving,"""Why give globally? Why give money? Why health...",www.jefftk.com,2016-12-07 05:00:00+00,2016-12-07 05:00:00+00,1,giving,custom,59,0
2,2,http://calnewport.com/blog/2013/03/24/how-to-w...,How to Write Six Important Papers a Year witho...,"""I'm fascinated by people who produce a large ...",calnewport.com,2016-12-08 05:00:00+00,2016-12-08 05:00:00+00,1,productivity,custom,56,0
3,3,http://www.freakonomics.com/2012/10/31/an-alte...,An Alternative to Democracy,"""In Glen's voting mechanism\, every voter can ...",www.freakonomics.com,2016-12-09 05:00:00+00,2016-12-09 05:00:00+00,1,politicalscience,custom,29,0
4,4,http://www.wired.com/vanish/2009/11/ff_vanish2...,Writer Evan Ratliff Tried to Vanish: Here's Wh...,"""August 13\, 6:40 PM: I'm driving East out of ...",www.wired.com,2016-12-10 05:00:00+00,2016-12-10 05:00:00+00,1,technology,custom,92,0
...,...,...,...,...,...,...,...,...,...,...,...,...
77470,134006,http://www.cleverhans.io/security/privacy/ml/2...,The challenge of verification and testing of m...,\N,\N,2021-10-05 23:00:44.806635,2021-10-05 23:00:44.806647,\N,\N,Custom,51,0
77471,134007,http://www.cleverhans.io/security/privacy/ml/2...,Is attacking machine learning easier than defe...,\N,\N,2021-10-05 23:00:44.815512,2021-10-05 23:00:44.815523,\N,\N,Custom,78,0
77472,134008,http://www.cleverhans.io/security/privacy/ml/2...,Breaking things is easy,\N,\N,2021-10-05 23:00:44.824349,2021-10-05 23:00:44.824360,\N,\N,Custom,86,0
77473,134009,https://docs.google.com/document/d/1kaHdXIqQ_7...,AID research proposals (4/2021) - Google Docs,\N,\N,2021-10-05 23:00:44.832961,2021-10-05 23:00:44.832973,\N,\N,Custom,33,0


In [8]:
links.to_csv('data/export.csv', index=False)